In [1]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


In [2]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv")


In [3]:
clean_df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%,caption_crop_middle_70%,captions_cn
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...,Shenzhen Bao'an M Nanshan Futian Luohu Car Dec...,塔那巴塔情人节 清心宝安 南山福田罗虎 汽车装饰花花花花花花 胸衣布置安排
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...,over 6 Years Old Girls 3-9 Boys 7 Students I D...,6岁以上的儿童玩具小女孩 3-9 男孩 7 学生I日 学生I日 10 儿童8 婴儿4 拼图
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...,Alternative Sex between Men and Women with Met...,使用金属尾狐尾巴的男女替代性行为
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...,Burnin' Up Yingkesong Landscape Painting Patro...,中华传统绘画烧在延克松露地绘画主办人 客厅绘画老板办公室 幸运装饰墙壁
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...,Men's Jeans Thick Section Distressed Straight ...,Lgnace Lee Lee Men的Jeans Thick分会受困于直立青年 秋秋潮的B...


In [4]:
# col_to_test =  [c for c in clean_df.columns.values if "caption" in c] + [c for c in clean_df.columns.values if "step" in c]
col_to_test =  ["caption"]

In [5]:
col_to_test

['caption']

In [6]:
image_size = 384
model_url = '/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_0.pth'

preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
clip_model =ALIPRODUCT_BLIP(model_url,image_size,vit='large')

load checkpoint from /home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_0.pth


In [7]:
def test_feature(caption_col):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,100,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    # print(image_embed.size())
    # print(text_embed.size())
    faiss_index = faiss.IndexFlatIP(256)
    print(image_embed.numpy().astype(np.float32).shape)
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    print(caption_col,"num correct pred top 5 k",sum(top5_preds))
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    print(caption_col,"num correct pred top 10 k",sum(top10_preds))

    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc

In [8]:
col_name = []
top5_acc_preds = []
top10_acc_preds = []
for col in col_to_test:
    top5_acc,top10_acc = test_feature(col)
    col_name.append(col)
    top5_acc_preds.append(top5_acc)
    top10_acc_preds.append(top10_acc)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 500/500 [51:41<00:00,  6.20s/it]
(50000, 256)
caption num correct pred top 5 k 19250
caption num correct pred top 10 k 25230


In [9]:
pred_df = {"caption_clean_method":col_name,"top_5":top5_acc_preds,"top_10":top10_acc_preds}

In [10]:
top5_acc_preds

[0.385]

In [11]:
pred_df = pd.DataFrame(pred_df)

In [12]:
pred_df

,caption_clean_method,top_5,top_10
0,caption,0.385,0.5046
